In [91]:
# importing libraries
import pandas as pd
import numpy as np
from ua_parser import user_agent_parser   # for user-agent string
from urllib.parse import unquote          # for non-english words
import urllib.parse
from geolite2 import geolite2
import sys
import re
# pd.set_option('display.max_columns', 1000)
# pd.set_option('display.max_colwidth', -1)

In [115]:
# encoding non-english words in native language
# def encode(url):
#     return urllib.parse.quote(url)

# function to extract locations from ip address city, country, continent, subdivisions 
def location(data, loc):
    geo = geolite2.reader()
    data[loc]= ''

    for i, ip in enumerate(data["geo_ip"]):
        x = geo.get(ip)
        try:
            data[loc].iloc[i] = x[loc]['names']['en']
        except:
            try:
                data[loc].iloc[i] = x[loc][0]['names']['en']
            except:
                pass
    geolite2.close()
    

# function to extract latitude, longitude, timezone etc...
def lat_long(data):
    data['latitude'] = ''
    data['longitude'] = ''
    data['time_zone'] = ''
    data['metro_code'] = ''
    
    geo = geolite2.reader()
    for i, ip in enumerate(data['geo_ip']):
        x= geo.get(ip)
        try:
            data['latitude'].iloc[i] = x["location"]['latitude']
            data['longitude'].iloc[i] = x["location"]['longitude']
        except:
            pass
        try:
            data['time_zone'].iloc[i] = x["location"]['time_zone']
        except:
            pass
        try:
            data['metro_code'].iloc[i] = x["location"]['metro_code']
        except:
            pass
    geolite2.close()
    
    
# function to extract information from 'location query' column
def extract(data, name):
    data[name] = ''
    for i, url in enumerate(data['location query']):
        try:
            data[name].iloc[i] = re.findall(f'{name}=(.*?)&', url)[0]
        except:
            try:
                data[name].iloc[i] = re.findall(f'{name}=(.*)', url)[0]
            except:
                pass
        

def main():

    # reading the I/O file
    data = pd.read_csv('sample_data.csv', encoding='utf-8')

    # applying non-english decoder function
#     data['location query']=data['location query'].apply(encode)


    # Handling User-agent column and making new columns for browser, os, devices

    ua = data['ua_string'].apply(lambda ua: user_agent_parser.Parse(ua))

    data['browser'] = ua.apply(lambda x: x['user_agent']['family'])
    data['os'] = ua.apply(lambda x: x['os']['family'])
    data['operating_device'] = ua.apply(lambda x: x['device']['family'])
    
    # extract details from the url
    extract(data,'creative')
    extract(data,'keyword')
    extract(data,'device')
    extract(data,'matchtype')
    extract(data,'n_media')
    extract(data,'n_campaign_type')
    extract(data,'gclid')
    extract(data,'n_query')
    extract(data,'n_keyword')
    extract(data,'n_ad_group')

    # applying function to extract geo info
    lat_long(data)

    # applying function to extract country details from geo_ip
    location(data, 'city')
    location(data, 'country')
    location(data, 'continent')
    location(data, 'subdivisions')
    
    data.replace('', np.nan, inplace=True)
    
    # Saving the updated file
    data.to_csv('Data.csv', index=False)
    
    # table with top 10 keywords extracted from URL
    url_feature = ['creative', 'keyword', 'device', 'matchtype', 'n_media', 'n_campaign_type', 'gclid', 'n_query', 'n_keyword', 'n_ad_group']

    feature=[]
    for features in url_feature:
        feature.append(data[features].value_counts().index[:10])

    data_value_count = pd.DataFrame(feature).T
    data_value_count.columns= url_feature
    
    # Saving the file 
    data_value_count.to_csv('Data_count.csv', index=False)


if __name__ == "__main__":
    main()